**Name: Debangshu Bhattacharya**

**Roll: MDS201910**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
#importing necessary library and packages
import numpy as np
import re
import os
from tqdm import tqdm
from collections import defaultdict
from nltk.util import ngrams
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import time
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#reading preprocessed_corpus
wd = 'My Drive/NLP/'
with open(os.path.join(wd, 'preprocessed_corpus.txt'), 'r') as f:
  corpus = f.read()

In [ ]:
#transforming the preprocessed corpus as a list of sentences. In my preprocessing I was separating each sentence by the start symbol '$$$$' and end symbol '$$$$' respectively.
#Also documents were separaated by '\n'. Here, I am handling that and transforming the corpus into a list of sentences.
sentences = []
docs = corpus.split('\n')
for doc in tqdm(docs):
  sentences_doc = doc.split('$$$$$$$$')
  t = sentences_doc[0]
  sentences_doc[0] = t[4:]

  t = sentences_doc[-1]
  sentences_doc[-1] = t[:-4]
  
  for sent in sentences_doc:
    sentences.append(sent)



100%|██████████| 54941/54941 [00:04<00:00, 12096.51it/s]


In [ ]:
# randomly selecting a subset of the corpus to work with. If I am performing the experiment on the whole corpus, my RAM is crashing.
# So, after experimenting on the value of the corpus to be chosen 
random.seed(42)
N = len(sentences)
subset_size = 700000
sentence_idxs = random.sample(range(N), subset_size)
sentence_subset = [sentences[i] for i in sentence_idxs]

len(sentence_subset)

700000

In [ ]:
from sklearn.model_selection import train_test_split
train_sentences, test_sentences = train_test_split(sentence_subset, test_size = 0.1)

In [ ]:
len(train_sentences), len(test_sentences)

(630000, 70000)

In [ ]:
del corpus

In [ ]:
del sentences

## Trigram model

In [ ]:
def build_trigram_model(sentences):
  #function to build a trigram model with a list of sentences as input
  model = defaultdict(lambda: defaultdict(lambda: 0))
  #iterating over sentences
  for sentence in tqdm(sentences):
    #tokenizing the sentence into a list of words
    tokens = word_tokenize(sentence)
    words = [word for word in tokens if word.isalpha()]
    #finding the trigrams for the words for the sentence
    trigrams = list(ngrams(words,3, pad_left = True, pad_right = True, left_pad_symbol = '<s>', right_pad_symbol = '</s>'))
    #updating frequency of each parameter
    for w1,w2,w3 in trigrams:
      model[(w1,w2)][w3]+=1

  #updating probability of each parameter    
  for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count
  return model

In [ ]:
#generate sentence based on trigram language model
def generate_sentence_from_trigram_model(model, first_word_argmax = False, deterministic = False):
  '''function to generate sentence with trigram model given as input. The parameter "first_word_argmax" is used to determine if the first word to be chosen is going to be the word which has maximum occurence.
  If first_word_argmax = True, then the model chooses the word which has the highest single occurence, else it chooses the first word randomly based on predicted probability of words.
  The parameter "deterministic" is used to determine if the language to be generated is deterministic or not.
  
  For the rest of the words to be generated, if the parameter "deterministic" is True, then we generate word (w3) based on the 
  maximum probability given the previous two words(w1,w2). If the parameter "deterministic" is False, then we generate a word randomly based on the probablity distribution of w3|w1,w2.   
  '''
  w1 = '<s>'
  w2 = '<s>'

  sent = ''
  words = list(model[(w1,w2)].keys())
  probs = list(model[(w1,w2)].values())

  if first_word_argmax == True:
    word = words[np.argmax(probs)]
  else:
    word = np.random.choice(words, p  = probs)
  
  first_word = word

  sent = sent + first_word
  sentence_finish = False
  while not sentence_finish:
    w1 = w2
    w2 = word

    words = list(model[(w1,w2)].keys())
    probs = list(model[(w1,w2)].values())

    if not deterministic:
      word = np.random.choice(words, p = probs)
    else:
      word = words[np.argmax(probs)]

    #print (word)
    if word=='</s>':
      sentence_finish = True
      break
    sent = sent + ' ' + str(word)

    
  return sent

**1) Language Model of trigram model**

In [ ]:
print ("Building trigram language model")

start_clock = time.time()
model = build_trigram_model(train_sentences)
end_clock = time.time()
print ("Model built")

print ("Time taken = %.3f minutes" %((end_clock-start_clock)/60))

  0%|          | 0/630000 [00:00<?, ?it/s]

Building trigram language model


100%|██████████| 630000/630000 [02:52<00:00, 3661.06it/s]


Model built
Time taken = 2.978 minutes


**2) Generating a sentence using trigram model**

Generating a sentence based on model parameters, i.e, choosing word w3 given w1 and w2. 

Based on the model's predicted probability of word w3 given w1 and w2, i.e,  P(w3|w1,w2), I am going to show how the model generates sentences over three variants of probability model. 

#### a) First word is the word with highest occurence in the corpus and rest of the words generated are one with the highest P(w3|w1,w2).

In [ ]:
print ("Generating sentence")
start_clock = time.time()
generated_sent = generate_sentence_from_trigram_model(model, first_word_argmax = True, deterministic=True)
end_clock = time.time()
print ("Generated sentence: %s" %generated_sent)
print ("Time taken = %.3f seconds" %((end_clock-start_clock)/60))


Generating sentence
Generated sentence: the copyright holder for this preprint this version posted may
Time taken = 0.000 seconds


#### b) Choosing first word randomly based on the probability distribution of P(first_word| start_token, start_token) and for the rest of the words selecting the word w3 with highest P(w3|w1,w2)

In [ ]:
print ("Generating sentence")
start_clock = time.time()
generated_sent = generate_sentence_from_trigram_model(model, first_word_argmax = False, deterministic=True)
end_clock = time.time()
print ("Generated sentence: %s" %generated_sent)
print ("Time taken = %.3f seconds" %((end_clock-start_clock)/60))


Generating sentence
Generated sentence: interdisciplinary simulations between obstetrics and gynecology university of california san francisco
Time taken = 0.001 seconds


#### c) Complete Probabilistic sentence generation with each word selected randomly based on the probability distribution of P(w3|w1,w2)

In [ ]:
print ("Generating sentence")
start_clock = time.time()
generated_sent = generate_sentence_from_trigram_model(model, first_word_argmax = False, deterministic=False)
end_clock = time.time()
print ("Generated sentence: %s" %generated_sent)
print ("Time taken = %.3f seconds" %((end_clock-start_clock)/60))


Generating sentence
Generated sentence: figure shows the corresponding period in the biosensor
Time taken = 0.001 seconds


Finding average probability of generating test sentences given trained three gram model parameters.

In [ ]:
def find_probability_sentence_trigram(model, sn):
  '''
  function to find probability of generation of a test sentence given parameters of trigram language model
  '''
  epsilon = 1e-10
  w1 = '<s>'
  w2 = '<s>'

  log_p_sum = 0
  tokens = word_tokenize(sn)
  words = [word for word in tokens if word.isalpha()]
  
  #appending end symbol to words
  words.append('</s>')
  words.append('</s>')
  for word in words:
    if word in model[(w1,w2)]: 
      p = model[(w1,w2)][word]
      #print ((w1,w2,word), p)
    else:
      p = epsilon
      #print ((w1, w2, word))
    log_p_sum += np.log(p)    

    w1 = w2
    w2 = word

  #print (len(words)+2, c)
  return np.exp(log_p_sum)



In [ ]:
def evaluate_trigram_model_on_test(model, test_sentences):
  #function to return the average probability of generating test corpus of sentences given the model parameters
  avg_prob = 0
  for sn in tqdm(test_sentences):
    prob_sentence = find_probability_sentence_trigram(model, sn)
    avg_prob += prob_sentence

  avg_prob = avg_prob/(len(test_sentences))
  return avg_prob

In [ ]:
avg_prob = evaluate_trigram_model_on_test(model, test_sentences)
print ("\nAverage probability of generating test corpus given three gram model parameters = %f" %avg_prob)

100%|██████████| 70000/70000 [00:22<00:00, 3118.01it/s]


Average probability of generating test corpus given three gram model parameters = 0.000026


In [ ]:
del model

## 4-gram model



In [ ]:
def build_4_gram_model(sentences):
  #function to return the four gram language model 
  model = defaultdict(lambda: defaultdict(lambda: 0))
  #iterating over sentences
  for sentence in tqdm(sentences):
    #tokenizing each sentence into words
    tokens = word_tokenize(sentence)
    words = [token for token in tokens if token.isalpha()]
    four_grams = list(ngrams(words,4, pad_left = True, pad_right = True, left_pad_symbol = '<s>', right_pad_symbol = '</s>'))
    #updating frequency of four gram tuple
    for w1,w2,w3,w4 in four_grams:
      model[(w1,w2,w3)][w4]+=1

  #updating probability of four gram tuple
  for w1_w2_w3 in model:
    total_count = float(sum(model[w1_w2_w3].values()))
    for w4 in model[w1_w2_w3]:
        model[w1_w2_w3][w4] /= total_count
  return model

In [ ]:
def generate_sentence_from_four_gram_model(model, first_word_argmax = False, deterministic = False):
  '''function to generate sentence with four gram model given as input. The parameter "deterministic" is used to determine if the language to be generated is deterministic or not.
  If the parameter "first_word_argmax" is True, then the first word generated is the word with the maximum probability. Else, the first word is also chosen based on model predicted word probability distribution.
  For the rest of the words to be generated, if the parameter "deterministic" is True, then we generate word (w4) based on the 
  maximum probability given the previous two words(w1,w2,w3). If the parameter "deterministic" is False, then we generate a word randomly based on the probablity distribution of w4|w1,w2,w3.   
  ''' 
  w1 = '<s>'
  w2 = '<s>'
  w3 = '<s>'
  
  sent = ''
  words = list(model[(w1,w2,w3)].keys())
  probs = list(model[(w1,w2,w3)].values())

  if first_word_argmax == True:
    word = words[np.argmax(probs)]
  else:
    word = np.random.choice(words, p = probs)
    
  first_word = word

  sent = sent + first_word
  sentence_finish = False
  while not sentence_finish:
    w1 = w2
    w2 = w3
    w3 = word

    words = list(model[(w1,w2,w3)].keys())
    probs = list(model[(w1,w2,w3)].values())

    if not deterministic:
      word = np.random.choice(words, p = probs)
    else:
      word = words[np.argmax(probs)]

    #print (word)
    if word=='</s>':
      sentence_finish = True
      break
    sent = sent + ' ' + str(word)

    
  return sent

**3) Build a language model using 4-grams**

In [ ]:
print ("\nBuilding 4-gram model")
start_clock = time.time()
model = build_4_gram_model(train_sentences)
end_clock = time.time()
print ("Model built")

print ("Time taken = %.3f minutes" %((end_clock-start_clock)/60))

  0%|          | 390/630000 [00:00<02:41, 3896.41it/s]


Building 4-gram model


100%|██████████| 630000/630000 [03:12<00:00, 3271.64it/s]


Model built
Time taken = 3.417 minutes


**4) Generating sentence using 4-gram model**

Just like for the sentence generation of the three gram model, I am also going to show how the model generates sentence based on three cases:

#### a) First word is the word with highest occurence in the corpus and rest of the words generated are one with the highest P(w4|w1,w2,w3).

In [ ]:
print ("Generating sentence")
start_clock = time.time()
generated_sent = generate_sentence_from_four_gram_model(model, first_word_argmax = True, deterministic=True)
end_clock = time.time()
print ("Generated sentence: %s" %generated_sent)
print ("Time taken = %.3f minutes" %((end_clock-start_clock)/60))

Generating sentence
Generated sentence: the copyright holder for this preprint this version posted may
Time taken = 0.000 minutes


#### b) First word is a word selected randomly based on model predicted P(first_word|start_token, start_token, start_token) and the rest of the words are chosen with highest P(w4|w1,w2,w3).

In [ ]:
print ("Generating sentence")
start_clock = time.time()
generated_sent = generate_sentence_from_four_gram_model(model, first_word_argmax = False, deterministic=True)
end_clock = time.time()
print ("Generated sentence: %s" %generated_sent)
print ("Time taken = %.3f minutes" %((end_clock-start_clock)/60))

Generating sentence
Generated sentence: these results suggest that the risk of infection
Time taken = 0.001 minutes


#### c) Complete probabilistic sentence where each word selected is chosen randomly based on model predicted P(w4|w1,w2,w3)

In [ ]:
print ("Generating sentence")
start_clock = time.time()
generated_sent = generate_sentence_from_four_gram_model(model, first_word_argmax = False, deterministic=False)
end_clock = time.time()
print ("Generated sentence: %s" %generated_sent)
print ("Time taken = %.3f minutes" %((end_clock-start_clock)/60))

Generating sentence
Generated sentence: mals is considered as the sum of viable eggs shed by infected humans
Time taken = 0.002 minutes


In [ ]:
def find_probability_sentence_four_gram(model, sn):
  '''
  function to find probability of generation of a test sentence given parameters of 4 gram language model. If a 4-gram (w1,w2,w3,w4) is not present in the training set, we replace that probability
  with epsilon which is a very small value and for our purpose we are choosing 10^-10.
  
  '''
  epsilon = 1e-10
  w1 = '<s>'
  w2 = '<s>'
  w3 = '<s>'

  log_p_sum = 0
  tokens = word_tokenize(sn)
  words = [word for word in tokens if word.isalpha()]
  
  
  words.append('</s>')
  words.append('</s>')
  words.append('</s>')

  for word in words:
    if word in model[(w1,w2,w3)]: 
      p = model[(w1,w2,w3)][word]
      #print ((w1,w2,word), p)
    else:
      p = epsilon
      #print ((w1, w2, word))
    log_p_sum += np.log(p)
        

    w1 = w2
    w2 = w3
    w3 = word

  #print (len(words)+2, c)
  return np.exp(log_p_sum)

In [ ]:
def evaluate_four_gram_model_on_test(model, test_sentences):
  #function to return the average probability of generating test corpus of sentences given the four gram model parameters
  avg_prob = 0
  for sn in tqdm(test_sentences):
    prob_sentence = find_probability_sentence_four_gram(model, sn)
    avg_prob += prob_sentence

  avg_prob = avg_prob/(len(test_sentences))
  return avg_prob

In [ ]:
avg_prob = evaluate_four_gram_model_on_test(model, test_sentences)
print ("\nAverage probability of generating test corpus given four gram model parameters = %f" %avg_prob)

100%|██████████| 70000/70000 [00:22<00:00, 3179.10it/s]


Average probability of generating test corpus given four gram model parameters = 0.000028


**5) The 4-gram model performed slightly better than the three gram langugage model. For each test sentence, I am finding the probability of the sentence based on the model parameters. Then I am averaging over the number of test sentences to find the average probability of test sentences based on model parameters. For 4-gram model, the average probability is slightly higher than the average probability of test sentences based on 3-gram model parameters.**

**6) Efficient ways to handle the large set of parameters:**

**a) One optimization that can be done to this is in handling the storage of tri-gram and 4-gram language models. Currently, I am storing the trigram model using a dictionary of dictionaries where the outer dictionary stores the tuple of words (w1, w2) as keys and the inner dictionary as value which is a dictionary with word w3 as key and P(w1,w2,w3) as value. Since, each character of a word takes one byte and the words can take any number of characters, the storage increases significantly. Here, we can have an integer mapping for the words and an inverse mapping from the integers to words as a separate map. While this takes an initial extra overhead in complexity, this will significantly improve storage complexity for the ngram models.**

**b) Another optimization that can be done is while computing the frequency of tuples (w1,w2,w3). We can separately perform this operation on batches of train data parallely and then combine them together.**

**7) Yes, it is possible to run some parts of the code in parallel. While building the ngram model, the first part involves computing the frequency of the tuple (w1, w2, w3,..,wn) for n grams w1,w2,...wn. Here, this frequency is done over the training sentences iteratively in a sequential manner. However, we can also perform this computation in batches in parallel and finally we can sum up the frequency values obtained for the same keys over batches to get the total frequency count. This would speed up building the model significantly.** 